# CNN Training

From our paper on "Explainable Prediction of Acute Myocardial Infarction using Machine Learning and Shapley Values"

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import pylab as plt
from keras import optimizers, losses, activations, models, regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, Flatten, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate
from keras.utils import to_categorical
from keras.models import load_model, Sequential
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


# Loading Data

In [2]:
# Load data
# Import train and test data into dataframes from csv files produced using the data processing code
df_cnn_train = pd.read_csv("train.csv", header=None)
df_cnn_train = df_cnn_train.sample(frac=1)
df_cnn_test = pd.read_csv("test.csv", header=None)

In [6]:
# Get data from dataframes
y_cnn = np.array(df_cnn_train[11].values).astype(np.int8)
y_cnn=to_categorical(y_cnn)
x_cnn = np.array(df_cnn_train[list(range(11))].values)[..., np.newaxis]
y_cnn_test = np.array(df_cnn_test[11].values).astype(np.int8)
x_cnn_test = np.array(df_cnn_test[list(range(11))].values)[..., np.newaxis]

# Model Definition + Training

In [4]:
# Model definition
def get_model(learning_rate=0.001):
    nclass = 2
    inp = Input(shape=(11, 1))
    
    cnn = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(inp)
    cnn = Dropout(rate=0.1)(cnn) 
    cnn = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(cnn)
    cnn = Dropout(rate=0.1)(cnn)
    cnn = Convolution1D(64, kernel_size=3, activation=activations.relu, padding="valid")(cnn)
    cnn = Dropout(rate=0.1)(cnn)
    cnn = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(cnn)
    cnn = GlobalMaxPool1D()(cnn)
    cnn = Dropout(rate=0.1)(cnn)
    dense_1 = Dense(64, activation=activations.relu, name="dense_1", kernel_regularizer=regularizers.l2(l=0.1))(cnn)
    dense_1 = Dense(16, activation=activations.relu, name="dense_2", kernel_regularizer=regularizers.l2(l=0.1))(dense_1)
    dense_1 = Dense(nclass, activation=activations.softmax, name="dense_3_ecg_view")(dense_1) 
    

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(learning_rate)

    model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=['acc'])
    return model

In [7]:
# Model Training
# Load model and model summary
model = get_model()
model.summary()

# File path to save the model
file_path = "cnn_ecgview.h5"

# Checkpoint the model's weight based on the accuracy of the model
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

# Set early stopping based on accuracy. It stops after 10 consecutive epochs of no accuracy improvement.
early = EarlyStopping(monitor="val_acc", mode="max", patience=10, verbose=1)

# Reduce learning rate based on accuracy. It reduces the rate after 7 consecutive epochs of no accuracy improvement.
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=7, verbose=2)

callbacks_list = [checkpoint, early, redonplat]

# Train the model, load weights into above file path to save the model
model.fit(x_cnn, y_cnn, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model.load_weights(file_path)

# The file will be saved in the file_path and can be loaded later using Keras for evaluation

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 11, 1)             0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 7, 16)             96        
_________________________________________________________________
dropout_5 (Dropout)          (None, 7, 16)             0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 5, 32)             1568      
_________________________________________________________________
dropout_6 (Dropout)          (None, 5, 32)             0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 3, 64)             6208      
_________________________________________________________________
dropout_7 (Dropout)          (None, 3, 64)             0   

 - 64s - loss: 0.3249 - acc: 0.8561 - val_loss: 0.2843 - val_acc: 0.8802

Epoch 00039: val_acc improved from 0.87994 to 0.88018, saving model to baseline_cnn_ecgview.h5
Epoch 40/1000
 - 60s - loss: 0.3254 - acc: 0.8559 - val_loss: 0.2930 - val_acc: 0.8748

Epoch 00040: val_acc did not improve from 0.88018
Epoch 41/1000
 - 57s - loss: 0.3245 - acc: 0.8559 - val_loss: 0.2832 - val_acc: 0.8833

Epoch 00041: val_acc improved from 0.88018 to 0.88330, saving model to baseline_cnn_ecgview.h5
Epoch 42/1000
 - 53s - loss: 0.3232 - acc: 0.8567 - val_loss: 0.2892 - val_acc: 0.8819

Epoch 00042: val_acc did not improve from 0.88330
Epoch 43/1000
 - 64s - loss: 0.3231 - acc: 0.8575 - val_loss: 0.2904 - val_acc: 0.8779

Epoch 00043: val_acc did not improve from 0.88330
Epoch 44/1000
 - 52s - loss: 0.3225 - acc: 0.8574 - val_loss: 0.2773 - val_acc: 0.8826

Epoch 00044: val_acc did not improve from 0.88330
Epoch 45/1000
 - 53s - loss: 0.3225 - acc: 0.8575 - val_loss: 0.2926 - val_acc: 0.8746

Epoch 00


Epoch 00093: val_acc did not improve from 0.89793
Epoch 94/1000
 - 56s - loss: 0.2923 - acc: 0.8723 - val_loss: 0.2495 - val_acc: 0.8982

Epoch 00094: val_acc improved from 0.89793 to 0.89819, saving model to baseline_cnn_ecgview.h5
Epoch 95/1000
 - 48s - loss: 0.2918 - acc: 0.8726 - val_loss: 0.2486 - val_acc: 0.8980

Epoch 00095: val_acc did not improve from 0.89819
Epoch 96/1000
 - 46s - loss: 0.2917 - acc: 0.8725 - val_loss: 0.2482 - val_acc: 0.8993

Epoch 00096: val_acc improved from 0.89819 to 0.89930, saving model to baseline_cnn_ecgview.h5
Epoch 97/1000
 - 52s - loss: 0.2920 - acc: 0.8720 - val_loss: 0.2499 - val_acc: 0.8970

Epoch 00097: val_acc did not improve from 0.89930
Epoch 98/1000
 - 60s - loss: 0.2920 - acc: 0.8725 - val_loss: 0.2468 - val_acc: 0.8996

Epoch 00098: val_acc improved from 0.89930 to 0.89958, saving model to baseline_cnn_ecgview.h5
Epoch 99/1000
 - 55s - loss: 0.2905 - acc: 0.8733 - val_loss: 0.2479 - val_acc: 0.8976

Epoch 00099: val_acc did not improve